In [1]:
! pip install openpyxl pandas numpy

Looking in indexes: https://artifactory.tcsbank.ru/artifactory/api/pypi/python-all/simple

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

channels = pd.read_excel("../data/tg_channels.xlsx")
#df = pd.read_csv("../data/cleaned_news_exp.csv")[["message_id", "id_channel", "message", "date", "topic"]]
df = pd.read_parquet("../data/tg_news_full.parquet")[["message_id", "id_channel", "message", "date"]]

/home/mlcore/conda/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
ch_map = (channels[["id", "name"]]
          .dropna()
          .assign(id=lambda x: pd.to_numeric(x["id"], errors="coerce"))
          .dropna(subset=["id"])
          .assign(id=lambda x: x["id"].astype(int))
          .set_index("id")["name"]
          .to_dict())

df = df.copy()
df["id_channel"] = pd.to_numeric(df["id_channel"], errors="coerce")
df["channel_name"] = df["id_channel"].map(ch_map).fillna(df["id_channel"].astype("Int64").astype(str))
df.head()

,message_id,id_channel,message,date,channel_name
0,275548,3,"Правительство Словакии обсудит меры, которые м...",2025-01-02 17:00:02,Риа Новости
1,275547,3,В ДТП с туристическим автобусом в Таиланде пос...,2025-01-02 16:40:53,Риа Новости
2,275546,3,Премьер Израиля Нетаньяху выписан из больницы ...,2025-01-02 16:20:12,Риа Новости
3,275545,3,Подозреваемый в подрыве автомобиля Tesla Cyber...,2025-01-02 15:54:29,Риа Новости
4,275543,3,Спецоперация. Обстановка и главные события на ...,2025-01-02 15:32:55,Риа Новости


In [4]:
import re
import numpy as np
import pandas as pd

def clean_news_text(t: str) -> str:
    t = t or ""
    t = re.sub(r"#\w+", " ", t)
    t = re.sub(r"[⚡️📈📉🇷🇺✅❗️🔥⬛ ⬜ ⚫ ⚪🔹]+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def ensure_datetime(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    d = df.copy()
    d[col] = pd.to_datetime(d[col], utc=True, errors="coerce")
    d = d.dropna(subset=[col])
    d["date_day"] = d[col].dt.floor("D")
    return d

df = ensure_datetime(df, "date")
df["message_id"] = df["message_id"].astype(str)
df["message"] = df["message"].fillna("").astype(str).map(clean_news_text)

In [5]:
df.head()

,message_id,id_channel,message,date,channel_name,date_day
0,275548,3,"Правительство Словакии обсудит меры, которые м...",2025-01-02 17:00:02+00:00,Риа Новости,2025-01-02 00:00:00+00:00
1,275547,3,В ДТП с туристическим автобусом в Таиланде пос...,2025-01-02 16:40:53+00:00,Риа Новости,2025-01-02 00:00:00+00:00
2,275546,3,Премьер Израиля Нетаньяху выписан из больницы ...,2025-01-02 16:20:12+00:00,Риа Новости,2025-01-02 00:00:00+00:00
3,275545,3,Подозреваемый в подрыве автомобиля Tesla Cyber...,2025-01-02 15:54:29+00:00,Риа Новости,2025-01-02 00:00:00+00:00
4,275543,3,Спецоперация. Обстановка и главные события на ...,2025-01-02 15:32:55+00:00,Риа Новости,2025-01-02 00:00:00+00:00


In [6]:
#! pip install rank_bm25

In [7]:
from rank_bm25 import BM25Okapi

def tokenize_ru(text: str):
    text = text.lower()
    text = re.sub(r"[^0-9a-zа-яё\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

corpus_tok = [tokenize_ru(t) for t in df["message"].tolist()]
bm25 = BM25Okapi(corpus_tok)

In [8]:
#! pip install faiss-cpu faiss-gpu-cu12 faiss-gpu-cu11

In [9]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/multilingual-e5-large" 
encoder = SentenceTransformer(MODEL_NAME, device="cuda")  
texts = df["message"].tolist()
doc_inputs = ["passage: " + t for t in texts]

In [10]:
# E_docs = encoder.encode(
#     doc_inputs,
#     batch_size=64,
#     show_progress_bar=True,
#     normalize_embeddings=True,
# ).astype(np.float32)

In [11]:
# import faiss

# dim = E_docs.shape[1]
# index = faiss.IndexFlatIP(dim)      
# index.add(E_docs)

In [12]:
# # save
# from pathlib import Path
# import numpy as np
# import pandas as pd
# import faiss
# import pickle

# OUT = Path("indexes")
# OUT.mkdir(parents=True, exist_ok=True)

# #rowmap
# rowmap = df[["message_id","date","date_day","id_channel","channel_name"]].copy()
# rowmap.to_parquet(OUT / "rowmap.parquet", index=False)

# np.save(OUT / "E_docs_e5_large.npy", E_docs)

# faiss.write_index(index, str(OUT / "faiss_e5_large.index"))

# with open(OUT / "bm25_corpus_tok.pkl", "wb") as f:
#     pickle.dump(corpus_tok, f)


In [13]:
# load
from pathlib import Path
import numpy as np
import pandas as pd
import faiss
import pickle

INP = Path("indexes")

rowmap = pd.read_parquet(INP / "rowmap.parquet")

E_docs = np.load(INP / "E_docs_e5_large.npy")
index = faiss.read_index(str(INP / "faiss_e5_large.index"))

with open(INP / "bm25_corpus_tok.pkl", "rb") as f:
    corpus_tok = pickle.load(f)

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(corpus_tok)


In [14]:
assert len(rowmap) == E_docs.shape[0]
assert len(rowmap) == index.ntotal

In [15]:
import numpy as np
import pandas as pd
import re

def snippet(t: str, n: int = 1000) -> str:
    return t[:n]

def _topk_indices_from_scores(scores: np.ndarray, k: int) -> np.ndarray:
    k = min(k, len(scores))
    if k <= 0:
        return np.array([], dtype=int)
    if k == len(scores):
        idx = np.argsort(-scores)
    else:
        idx = np.argpartition(-scores, k-1)[:k]
        idx = idx[np.argsort(-scores[idx])]
    return idx.astype(int)

def dense_candidates_faiss(index, encoder, query: str, topN: int = 500):
    qv = encoder.encode(["query: " + query], normalize_embeddings=True, show_progress_bar=False).astype(np.float32)
    scores, idx = index.search(qv, topN) 
    return idx[0].astype(int), scores[0].astype(np.float32)


def hybrid_retrieve_rrf(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    k: int = 50,
    topN_each: int = 500,
    k_rrf: int = 60,
    w_dense: float = 1.0,
    w_bm25: float = 1.0,
    exclude_message_id: str | None = None,
    anchor_date: str | pd.Timestamp | None = None,
    date_col: str = "date_day") -> pd.DataFrame:

    if anchor_date is not None:
        ad = pd.to_datetime(anchor_date, utc=True).normalize()

        if date_col not in df.columns:
            raise KeyError(f"date_col='{date_col}' not found in df.columns")

        dts = pd.to_datetime(df[date_col], errors="coerce", utc=True).dt.normalize()
        allowed = (dts <= ad)
        allowed_np = allowed.to_numpy(dtype=bool)
    else:
        allowed_np = None

    d_idx, d_sc = dense_candidates_faiss(index, encoder, query, topN=topN_each)

    if allowed_np is not None:
        keep = allowed_np[d_idx]
        d_idx = d_idx[keep]
        d_sc = d_sc[keep]

    dense_rank = {int(rowpos): r for r, rowpos in enumerate(d_idx, start=1)}

    if bm25 is None:
        out = df.iloc[d_idx].copy()
        out["_rowpos"] = d_idx
        out["score_rrf"] = (w_dense / (k_rrf + out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan)).astype(np.float32)))
        out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
        out["rank_bm25"] = np.nan

        if exclude_message_id is not None and "message_id" in out.columns:
            out = out[out["message_id"].astype(str) != str(exclude_message_id)]

        return out.head(k).reset_index(drop=True)

    bm_scores = bm25.get_scores(tokenize_fn(query)).astype(np.float32)
    if allowed_np is not None:
        bm_scores[~allowed_np] = -np.inf

    b_idx = _topk_indices_from_scores(bm_scores, topN_each)
    bm_rank = {int(rowpos): r for r, rowpos in enumerate(b_idx, start=1)}

    union = np.array(sorted(set(dense_rank) | set(bm_rank)), dtype=int)
    rrf = np.zeros(len(union), dtype=np.float32)

    for j, rowpos in enumerate(union):
        if rowpos in dense_rank:
            rrf[j] += w_dense / (k_rrf + dense_rank[rowpos])
        if rowpos in bm_rank:
            rrf[j] += w_bm25 / (k_rrf + bm_rank[rowpos])

    order = np.argsort(-rrf)
    union = union[order]
    rrf = rrf[order]

    out = df.iloc[union].copy()
    out["_rowpos"] = union
    out["score_rrf"] = rrf
    out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
    out["rank_bm25"] = out["_rowpos"].map(lambda rp: bm_rank.get(int(rp), np.nan))

    if exclude_message_id is not None and "message_id" in out.columns:
        out = out[out["message_id"].astype(str) != str(exclude_message_id)]

    return out.head(k).reset_index(drop=True)



In [16]:
df["date_day"].min(), df["date_day"].max()

(Timestamp('2023-09-23 00:00:00+0000', tz='UTC'),
 Timestamp('2025-09-08 00:00:00+0000', tz='UTC'))

In [17]:
# ! pip install transformers torch sentence-transformers accelerate vllm

In [18]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# import torch

# MODEL = "Qwen/Qwen2.5-14B-Instruct"

# tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     MODEL,
#     torch_dtype=torch.bfloat16,
#     device_map="cuda",
#     trust_remote_code=True,
# )
# model.eval()

In [19]:
from transformers import AutoTokenizer, AutoConfig
from vllm import LLM, SamplingParams
import torch

MODEL = "Qwen/Qwen2.5-32B-Instruct"
#MODEL = "mistralai/Mistral-Small-3.1-24B-Instruct-2503"

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

cfg = AutoConfig.from_pretrained(MODEL, trust_remote_code=True)
native_ctx = getattr(cfg, "max_position_embeddings", None)
print("GPU:", torch.cuda.get_device_name(0))
print("VRAM(GB):", round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 1))
print("Model max_position_embeddings:", native_ctx)

MAX_MODEL_LEN = 24576


model = LLM(
    model=MODEL,
    dtype="bfloat16",
    max_model_len=MAX_MODEL_LEN,
    gpu_memory_utilization=0.90,
)

GPU: NVIDIA A100-SXM4-80GB
VRAM(GB): 79.3
Model max_position_embeddings: 32768
INFO 01-11 00:05:04 [utils.py:253] non-default args: {'dtype': 'bfloat16', 'max_model_len': 24576, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-32B-Instruct'}
INFO 01-11 00:05:05 [model.py:514] Resolved architecture: Qwen2ForCausalLM
INFO 01-11 00:05:05 [model.py:1661] Using max model len 24576
INFO 01-11 00:05:06 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 01-11 00:05:07 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
(EngineCore_DP0 pid=6421) INFO 01-11 00:05:14 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='Qwen/Qwen2.5-32B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-32B-Instruct', skip_tokenizer_init=Fal

(EngineCore_DP0 pid=6421) /home/mlcore/conda/lib/python3.10/site-packages/tvm_ffi/_optional_torch_c_dlpack.py:174: UserWarning: Failed to JIT torch c dlpack extension, EnvTensorAllocator will not be enabled.
(EngineCore_DP0 pid=6421) We recommend installing via `pip install torch-c-dlpack-ext`
(EngineCore_DP0 pid=6421)   warnings.warn(


(EngineCore_DP0 pid=6421) INFO 01-11 00:05:19 [cuda.py:351] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')


Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   6% Completed | 1/17 [00:00<00:09,  1.75it/s]
Loading safetensors checkpoint shards:  12% Completed | 2/17 [00:01<00:09,  1.58it/s]
Loading safetensors checkpoint shards:  18% Completed | 3/17 [00:01<00:09,  1.52it/s]
Loading safetensors checkpoint shards:  24% Completed | 4/17 [00:02<00:08,  1.50it/s]
Loading safetensors checkpoint shards:  29% Completed | 5/17 [00:03<00:07,  1.51it/s]
Loading safetensors checkpoint shards:  35% Completed | 6/17 [00:03<00:07,  1.51it/s]
Loading safetensors checkpoint shards:  41% Completed | 7/17 [00:04<00:06,  1.43it/s]
Loading safetensors checkpoint shards:  47% Completed | 8/17 [00:05<00:05,  1.53it/s]
Loading safetensors checkpoint shards:  53% Completed | 9/17 [00:05<00:05,  1.54it/s]
Loading safetensors checkpoint shards:  59% Completed | 10/17 [00:06<00:04,  1.51it/s]
Loading safetensors checkpoint shards:  65% Completed | 11/17

(EngineCore_DP0 pid=6421) INFO 01-11 00:05:31 [default_loader.py:308] Loading weights took 11.34 seconds
(EngineCore_DP0 pid=6421) INFO 01-11 00:05:32 [gpu_model_runner.py:3659] Model loading took 61.0375 GiB memory and 15.536622 seconds
(EngineCore_DP0 pid=6421) INFO 01-11 00:05:44 [backends.py:643] Using cache directory: /home/mlcore/.cache/vllm/torch_compile_cache/956b349265/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=6421) INFO 01-11 00:05:44 [backends.py:703] Dynamo bytecode transform time: 11.88 s
(EngineCore_DP0 pid=6421) INFO 01-11 00:05:51 [backends.py:261] Cache the graph of compile range (1, 8192) for later use
(EngineCore_DP0 pid=6421) INFO 01-11 00:06:08 [backends.py:278] Compiling a graph for compile range (1, 8192) takes 17.07 s
(EngineCore_DP0 pid=6421) INFO 01-11 00:06:08 [monitor.py:34] torch.compile takes 28.95 s in total
(EngineCore_DP0 pid=6421) INFO 01-11 00:06:09 [gpu_worker.py:375] Available KV cache memory: 8.81 GiB
(EngineCore_DP0 pid=6421) 

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:06<00:00,  7.60it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:03<00:00,  9.77it/s]


(EngineCore_DP0 pid=6421) INFO 01-11 00:06:20 [gpu_model_runner.py:4587] Graph capturing finished in 11 secs, took 4.11 GiB
(EngineCore_DP0 pid=6421) INFO 01-11 00:06:20 [core.py:259] init engine (profile, create kv cache, warmup model) took 48.70 seconds
INFO 01-11 00:06:22 [llm.py:360] Supported tasks: ['generate']


In [20]:
SYSTEM_PROMPT = """Ты — очень внимательный новостной аналитик. Ты пишешь аккуратный дайджест по новостям в стиле корреспондента.

Вход: запрос, актуальная дата (YYYY-MM-DD) и документы вида:
[id] date=YYYY-MM-DD channel(s)=<канал1; канал2; ...>
<текст>
Все документы датированы НЕ ПОЗЖЕ актуальной даты.

ОБЩИЕ ПРАВИЛА:
1) Пиши на русском. Используй ТОЛЬКО информацию из документов.
2) Не упоминай номера документов ([id]).
3) Не добавляй дат, которых нет в документах. Не используй “сегодня/вчера/недавно” — только YYYY-MM-DD.
4) Любые числа/точные значения/пороги/цитаты/формулировки — только если они есть в документах.
5) Каналы указывай ТОЛЬКО в дайджесте (в таймлайне каналов не будет).
6) Если рядом с актуальной датой мало данных — честно укажи последнюю дату в подборке и разрыв, без домыслов “что сейчас”.
7) Нельзя вставлять “пустые” даты и строки вида “нет данных”.

КАК ПИСАТЬ ДАЙДЖЕСТ:
- Начни с 2–4 самых свежих УНИКАЛЬНЫХ дат в подборке (это “последнее время”).
- Для каждого ключевого факта укажи источник:
  “Канал(ы) (YYYY-MM-DD): …”
- Не смешивай разные типы сигналов как одно и то же: явно различай
  “официальный курс / внебиржевой курс / расчетный курс / индекс доллара / прогноз”.
- Если встречаются повторы — объединяй.
- Пиши официально и нейтрально, как в новостном дайджесте (без эмоциональных оценок и без домыслов).

СТРУКТУРА ОТВЕТА (3 блока):

### 1) Запрос и актуальная дата
* Запрос: ...
* Актуальная дата: ...

### 2) Дайджест
Связный текст (8–20 предложений), как корреспондент:
- “Последнее время”: несколько ключевых фактов по самым свежим датам + кто сообщил.
- Затем коротко “раньше/предыстория”: несколько ключевых фактов по более ранним датам + кто сообщил.
- В конце 1–2 предложения: выводы, исходя из актуальности новостей. Можно ли дать актуальную оценку или нет. Насколько тема в целом актуальная, свежая и вирусная.

### 3) Таймлайн (полный, 1 дата = 1 строка = 1 факт, без каналов)
Полный список ВСЕХ УНИКАЛЬНЫХ дат из документов, в порядке от самой старой к самой новой (ascending).
Одна строка = одна (1) дата.
Одна строка = один (1) самый релевантный запросу факт/изменение, который явно есть в документах этой даты.

Формат строки:
* YYYY-MM-DD — что сообщили / что изменилось (1 факт)

ПРАВИЛА ТАЙМЛАЙНА (критично):
- В таймлайне НЕ УКАЗЫВАЙ каналы вообще.
- Не добавляй дат, которых нет в документах!
- Не переноси факты между датами “по смыслу”: в строке даты могут быть только факты из документов этой даты!
- Никаких причин и выводов — только “что сообщили”!
- Если на одну дату приходится несколько фактов, выбери один самый важный для запроса (остальное оставь в дайджесте)!
! ЕСЛИ ДАТЫ НЕТ ВНУТРИ КОНТЕКСТА — ЕЁ НЕЛЬЗЯ УКАЗЫВАТЬ. ЕСЛИ ФАКТА НЕТ НА ЭТУ ДАТУ В ДОКУМЕНТАХ — ЕГО НЕЛЬЗЯ ПИСАТЬ. !
"""



def build_rag_context(query: str, cand: pd.DataFrame, anchor_date: str, k_docs: int = 30, snip_chars: int = 850) -> str:
    c = cand.head(k_docs).copy()

    blocks = []
    for i, row in enumerate(c.itertuples(index=False), start=1):
        date_day = getattr(row, "date_day", getattr(row, "date", ""))
        if isinstance(date_day, pd.Timestamp):
            date_day = date_day.strftime("%Y-%m-%d")
        date_day = str(date_day)[:10]

        channel = getattr(row, "channel_name")
        text = getattr(row, "message", "")

        blocks.append(f"[{i}] date={date_day} channel(s)={channel}\n document=" + snippet(str(text), snip_chars))

    return (
        f"АКТУАЛЬНАЯ ДАТА ОБЗОРА: {anchor_date}\n"
        f"ВОПРОС/ЗАПРОС:\n{query}\n\n"
        f"ИСТОЧНИКИ:\n" + "\n\n".join(blocks)
    )


In [21]:
import re, json
import pandas as pd
from vllm import SamplingParams

JUDGE_SYSTEM = """Ты — строгий эксперт по информационному поиску по новостям (в т.ч. экономическим).

Твоя задача: оценить релевантность кандидатной новости запросу. Запрос может быть:
- коротким топиком (например "курс рубля к доллару"),
- или текстом другой новости (тогда запрос описывает конкретный инфоповод).

Используй ТОЛЬКО текст кандидатного документа. Ничего не додумывай.

Шкала релевантности:
2 — документ явно про то же самое: отвечает топику ИЛИ описывает тот же инфоповод/факт/событие, что и запрос.
1 — документ связан по теме/контексту, но это немного другой инфоповод, или про то же, но без прямого соответствия.
0 — нерелевантно совсем.

Правило строгости:
ставь 2 только если связь очевидна по тексту документа; если информации недостаточно — ставь 0 или 1.

Верни строго валидный JSON и ничего больше:
{"relevance": 0|1|2}
"""


def _parse_relevance(text: str) -> int:
    text = text.strip()
    m = re.search(r"\{.*\}", text, flags=re.DOTALL)
    if m:
        blob = m.group(0)
        try:
            obj = json.loads(blob)
            val = int(obj.get("relevance", 0))
            return val if val in (0, 1, 2) else 0
        except Exception:
            pass
    m2 = re.search(r"relevance\"\s*:\s*([012])", text)
    if m2:
        return int(m2.group(1))
    return 0

def judge_filter_candidates(
    cand: pd.DataFrame,
    query: str,
    judge_llm,
    judge_tokenizer,
    *,
    keep_threshold: int = 1,     
    doc_max_chars: int = 1200,
    batch_size: int = 32,
    max_out_tokens: int = 40,
) -> pd.DataFrame:
    if cand is None or len(cand) == 0:
        return cand

    text_col = "message"
    channel_col = "channel_name"
    date_col = "date_day"

    prompts = []
    for _, row in cand.iterrows():
        doc = str(row[text_col])[:doc_max_chars]
        ch = str(row[channel_col]) if channel_col else ""
        dt = str(row[date_col]) if date_col else ""

        user_msg = (
            f"ЗАПРОС:\n{query}\n\n"
            f"КАНДИДАТ:\n"
            f"channel={ch}\n"
            f"date={dt}\n"
            f"text:\n{doc}\n"
        )

        messages = [
            {"role": "system", "content": JUDGE_SYSTEM},
            {"role": "user", "content": user_msg},
        ]
        prompt = judge_tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        prompts.append(prompt)

    sampling = SamplingParams(
        temperature=0.0,
        top_p=1.0,
        max_tokens=max_out_tokens,
    )

    relevances = []
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i+batch_size]
        outs = judge_llm.generate(batch_prompts, sampling)
        for o in outs:
            txt = o.outputs[0].text
            relevances.append(_parse_relevance(txt))

    out_df = cand.copy()
    out_df["judge_relevance"] = relevances

    filtered = out_df[out_df["judge_relevance"] >= keep_threshold].copy()
    filtered.reset_index(drop=True, inplace=True)
    return filtered


In [22]:
# import torch

# @torch.inference_mode()
# def rag_summarize(sum_model, sum_tokenizer, query: str, cand: pd.DataFrame, anchor_date, 
#                   k_docs: int = 25, snip_chars: int = 900, max_new_tokens: int = 2000) -> str:
    
#     user = build_rag_context(query, cand, anchor_date=anchor_date, k_docs=k_docs, snip_chars=snip_chars)
#     print("built context...")
    
#     messages = [
#         {"role": "system", "content": SYSTEM_PROMPT},
#         {"role": "user", "content": user},
#     ]
#     prompt = sum_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     enc = sum_tokenizer(prompt, return_tensors="pt", truncation=True).to(sum_model.device)

#     out_ids = sum_model.generate(
#         **enc,
#         max_new_tokens=max_new_tokens,
#         do_sample=False,
#         eos_token_id=sum_tokenizer.eos_token_id,
#         pad_token_id=sum_tokenizer.eos_token_id,
#     )
#     prompt_len = int(enc["attention_mask"][0].sum().item())
    
#     return sum_tokenizer.decode(out_ids[0][prompt_len:], skip_special_tokens=True).strip(), user


In [23]:
from vllm import SamplingParams

def rag_summarize(sum_model, sum_tokenizer, query: str, cand: pd.DataFrame, anchor_date,
                  k_docs: int = 25, snip_chars: int = 900, max_new_tokens: int = 2000):

    user = build_rag_context(query, cand, anchor_date=anchor_date, k_docs=k_docs, snip_chars=snip_chars)
    print("built context...")

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user},
    ]

    prompt = sum_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )
    #Mistral
    #prompt = f"<s>[SYSTEM_PROMPT]{SYSTEM_PROMPT}[/SYSTEM_PROMPT][INST]{user}[/INST]"

    sampling = SamplingParams(
        temperature=0.0,  
        max_tokens=max_new_tokens,
        top_p=1.0,
    )

    result = sum_model.generate([prompt], sampling)[0]
    text = result.outputs[0].text.strip()

    return text, user


In [24]:
def run_rag_hybrid(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    exclude_message_id: str | None = None,
    k_retrieve: int = 50,
    topN_each: int = 500,
    k_docs: int = 25,
    snip_chars: int = 1500,
    max_new_tokens: int = 2000,
    anchor_date: str = "2025-09-04",
    
    sum_model=None,
    sum_tokenizer=None,

    judge_llm=None,
    judge_tokenizer=None,
    judge_keep_threshold: int = 1,
    judge_batch_size: int = 32,
):
    cand = hybrid_retrieve_rrf(
        df=df,
        index=index,
        encoder=encoder,
        bm25=bm25,
        tokenize_fn=tokenize_fn,
        query=query,
        k=k_retrieve,
        topN_each=topN_each,
        k_rrf=60,
        w_dense=1.0,
        w_bm25=1.0,
        exclude_message_id=exclude_message_id,
        anchor_date=anchor_date
    )

    cand_before = cand
    if judge_llm is not None and judge_tokenizer is not None and cand is not None and len(cand) > 0:
        cand = judge_filter_candidates(
            cand=cand,
            query=query,
            judge_llm=judge_llm,
            judge_tokenizer=judge_tokenizer,
            keep_threshold=judge_keep_threshold,
            doc_max_chars=snip_chars,
            batch_size=judge_batch_size,
        )

    if sum_model is None or sum_tokenizer is None:
        ctx = build_rag_context(query, cand, anchor_date=anchor_date, k_docs=k_docs, snip_chars=snip_chars)
        return {
            "context": ctx,
            "candidates": cand_before,
            "candidates_filtered": cand,
            "summary": "No LLM",
        }

    summary, ctx = rag_summarize(
        sum_model, sum_tokenizer,
        query, cand,
        k_docs=min(k_docs, len(cand)) if cand is not None else 0,
        snip_chars=snip_chars,
        max_new_tokens=max_new_tokens,
        anchor_date=anchor_date
    )

    return {
        "context": ctx,
        "summary": summary,
        "candidates": cand_before,
        "candidates_filtered": cand,
    }


In [25]:
q = "Актуальный курс доллара"

out = run_rag_hybrid(
    df=df,
    index=index,
    encoder=encoder,
    bm25=bm25,
    tokenize_fn=tokenize_ru,
    query=q,

    k_retrieve=150,
    topN_each=2000,
    k_docs=30,
    snip_chars=1500,
    max_new_tokens=5000,
    anchor_date="2025-09-04",

    sum_model=model,
    sum_tokenizer=tokenizer,

    judge_llm=model,
    judge_tokenizer=tokenizer,
    judge_keep_threshold=1,  
    judge_batch_size=32)


Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

...

Adding requests:   0%|          | 0/22 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/22 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

built context...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [26]:
from IPython.display import display, Markdown, HTML
import re

def show_summary(summary: str):
    if summary is None:
        display(HTML("<b>summary is None</b>"))
        return

    s = str(summary)
    s = s.replace("\\n", "\n") 
    s = re.sub(r"\n{3,}", "\n\n", s).strip()

    display(Markdown(s))

show_summary(out["summary"])


### 1) Запрос и актуальная дата
* Запрос: Актуальный курс доллара
* Актуальная дата: 2025-09-04

### 2) Дайджест
Последнее время, с 2025-06-30, индекс доллара опустился ниже 97 пунктов впервые с марта 2022 года, согласно данным Риа Новости. 2025-05-29, официальный курс доллара от ЦБ опустился ниже 79 рублей, составив 78,5 рубля, как сообщили Риа Новости. Ранее, 2025-04-21, ЦБ установил курс доллара ниже 81 рубля, до 80.7597 рубля, согласно Сигналам РЦБ. В 2025-03-17, расчетный курс доллара упал ниже 83 рублей, до 82,87 рубля, как сообщили "Раньше всех. Ну почти". В 2025-02-25, ЦБ понизил курс доллара до 86 рублей, согласно "Банки, деньги, два офшора". В 2024-12-06, ЦБ понизил курс доллара ниже 100 рублей, как сообщили "Банки, деньги, два офшора". В 2024-11-14, курс доллара превысил 100 рублей, согласно данным "Экономика". В 2024-04-19, некоторые экономисты прогнозировали, что курс доллара к июню превысит 100 рублей и может достичь 120 рублей, согласно Блумберг. 

Тема курса доллара остается актуальной и вирусной, однако последние данные о курсе доллара датированы 2025-06-30, что указывает на разрыв в информации с актуальной датой 2025-09-04. 

### 3) Таймлайн
* 2024-04-19 — некоторые экономисты прогнозировали, что курс доллара к июню превысит 100 рублей и может достичь 120 рублей
* 2024-04-26 — курс доллара – 87.87₽
* 2024-06-13 — курс доллара упал до 87₽
* 2024-11-14 — курс доллара превысил 100 рублей
* 2024-11-19 — официальный курс доллара превысил 100 рублей
* 2024-11-22 — курс доллара установлен выше 102 рублей
* 2024-11-27 — курс доллара перевалил за 111 рублей, затем до 114 рублей
* 2024-11-28 — курс доллара опустился ниже 110 рублей
* 2024-12-05 — курс доллара установлен на уровне 103,3
* 2024-12-06 — курс доллара понизился ниже 100 рублей
* 2024-12-11 — курс доллара поднят выше 103 рублей
* 2024-12-24 — курс доллара упал ниже 100 рублей
* 2025-02-25 — курс доллара понизился до 86 рублей
* 2025-03-11 — курс доллара опустился до 86 рублей
* 2025-03-17 — расчетный курс доллара упал ниже 83 рублей
* 2025-04-14 — курс доллара понизился более чем на 1 рубль
* 2025-04-21 — курс доллара установлен ниже 81 рубля
* 2025-05-21 — курс доллара установлен на уровне 79,75 рубля
* 2025-05-29 — курс доллара опустился ниже 79 рублей
* 2025-06-19 — оптимальный курс в текущей ситуации в районе 100 рублей за доллар
* 2025-06-20 — равновесным курсом является выше 100 рублей за доллар
* 2025-06-30 — индекс доллара опустился ниже 97 пунктов
* 2025-07-28 — курс доллара от ЦБ оказался меньше внебиржевого на полтора рубля
* 2025-08-30 — курс доллара опустился ниже 81 рубля

In [27]:
out["context"]

'АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04\nВОПРОС/ЗАПРОС:\nАктуальный курс доллара\n\nИСТОЧНИКИ:\n[1] date=2024-11-27 channel(s)=Блумберг\n document=Курс доллара перевалил за 111 рублей. UPD: Курс уже 114. 🔵 Bloomberg\n\n[2] date=2025-07-28 channel(s)=Банки, деньги, два офшора\n document=Свежий курс доллара от ЦБ оказался меньше внебиржевого на полтора рубля. @bankrollo\n\n[3] date=2024-12-06 channel(s)=Банки, деньги, два офшора\n document=ЦБ понизил курс доллара ниже 100, евро — ниже 107. @bankrollo\n\n[4] date=2025-04-21 channel(s)=Сигналы РЦБ\n document=ЦБ установил курс доллара ниже 81 рубля впервые с лета 2023 года ЦБ РФ с 22 апреля снизил официальный курс доллара США на 37.74 коп., до 80.7597 руб.\n\n[5] date=2024-11-14 channel(s)=Экономика \n document=Курс доллара теперь стоит выше 100 рублей. Это следует из данных Мосбиржи. «Стоимость доллара — 100,2428 рубля», — сообщает регулятор. Такого скачка не было несколько месяцев. Сейчас Центробанк сам регулирует цену доллара по отношению к 

In [33]:
import pandas as pd

def load_queries_from_parquet(parquet_path: str):
    ev = pd.read_parquet(parquet_path)

    if "query" not in ev.columns or "query_type" not in ev.columns:
        raise KeyError("parquet must contain columns: query, query_type")

    items = []
    for r in ev.itertuples(index=False):
        q = getattr(r, "query", None)
        qt = getattr(r, "query_type", None)
        if q is None or qt is None:
            continue
        q = str(q).strip()
        qt = str(qt).strip()
        if not q:
            continue

        items.append({
            "row_id": getattr(r, "row_id", None),
            "anchor_date": str(getattr(r, "anchor_date", "")) if hasattr(r, "anchor_date") else "",
            "query_type": qt,
            "query": q,
            "seed_message_id": getattr(r, "seed_message_id", None),
        })

    counts = {
        "topics_available": int((ev["query_type"].astype(str) == "topic").sum()),
        "news_available": int((ev["query_type"].astype(str) == "news").sum()),
        "topics_used": sum(1 for x in items if x["query_type"] == "topic"),
        "news_used": sum(1 for x in items if x["query_type"] == "news"),
        "all_used": len(items),
    }

    return items, counts, ev

items, counts, data = load_queries_from_parquet("eval_df_temporalrag.parquet")
print(counts)
print(items[0])


{'topics_available': 168, 'news_available': 32, 'topics_used': 168, 'news_used': 32, 'all_used': 200}
{'row_id': 0, 'anchor_date': '2025-09-04', 'query_type': 'topic', 'query': 'Актуальный курс доллара', 'seed_message_id': None}


In [37]:
import pandas as pd
from vllm import SamplingParams

def run_rag_hybrid_batch(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    sum_model,
    sum_tokenizer,
    items: list[dict],
    k_retrieve: int = 50,
    topN_each: int = 500,
    k_docs: int = 25,
    snip_chars: int = 1500,
    max_new_tokens: int = 2000,
    anchor_date_default: str = "2025-09-04",
    judge_llm=None,
    judge_tokenizer=None,
    judge_keep_threshold: int = 1,
    judge_batch_size: int = 32,
    gen_batch_size: int = 8,
):
    rows = []
    prompts = []
    prompt_row_ids = []

    for it in items:
        row_id = it.get("row_id", None)
        query = str(it.get("query", "")).strip()
        if not query:
            continue

        anchor_date = str(it.get("anchor_date", "")).strip() or anchor_date_default
        seed_message_id = it.get("seed_message_id", None)
        exclude_message_id = None if seed_message_id is None else str(seed_message_id)

        cand = hybrid_retrieve_rrf(
            df=df,
            index=index,
            encoder=encoder,
            bm25=bm25,
            tokenize_fn=tokenize_fn,
            query=query,
            k=k_retrieve,
            topN_each=topN_each,
            k_rrf=60,
            w_dense=1.0,
            w_bm25=1.0,
            exclude_message_id=exclude_message_id,
            anchor_date=anchor_date,
        )
        cand_before = cand

        if (
            judge_llm is not None
            and judge_tokenizer is not None
            and cand is not None
            and len(cand) > 0
        ):
            cand = judge_filter_candidates(
                cand=cand,
                query=query,
                judge_llm=judge_llm,
                judge_tokenizer=judge_tokenizer,
                keep_threshold=judge_keep_threshold,
                doc_max_chars=snip_chars,
                batch_size=judge_batch_size,
            )

        ctx = build_rag_context(
            query=query,
            cand=cand,
            anchor_date=anchor_date,
            k_docs=min(k_docs, len(cand)) if cand is not None else 0,
            snip_chars=snip_chars,
        )

        rid = row_id if row_id is not None else len(rows)

        rows.append({
            "row_id": rid,
            "anchor_date": anchor_date,
            "query": query,
            "query_type": it.get("query_type", None),
            "seed_message_id": seed_message_id,
            "exclude_message_id": exclude_message_id,
            "k_retrieve": k_retrieve,
            "topN_each": topN_each,
            "k_docs": k_docs,
            "snip_chars": snip_chars,
            "max_new_tokens": max_new_tokens,
            "context": ctx,
            "summary": None,
            "n_cand_before": int(len(cand_before)) if cand_before is not None else 0,
            "n_cand_final": int(len(cand)) if cand is not None else 0,
        })

        if sum_model is not None and sum_tokenizer is not None and cand is not None and len(cand) > 0:
            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": ctx},
            ]
            prompt = sum_tokenizer.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
            prompts.append(prompt)
            prompt_row_ids.append(rid)

    out_df = pd.DataFrame(rows)

    if len(prompts) == 0:
        return out_df

    sampling = SamplingParams(
        temperature=0.0,
        top_p=1.0,
        max_tokens=max_new_tokens,
    )

    summaries = []
    for start in range(0, len(prompts), int(gen_batch_size)):
        chunk = prompts[start:start + int(gen_batch_size)]
        outs = sum_model.generate(chunk, sampling)
        for o in outs:
            summaries.append(o.outputs[0].text.strip())

    rid_to_summary = {rid: s for rid, s in zip(prompt_row_ids, summaries)}
    out_df["summary"] = out_df["row_id"].map(rid_to_summary)

    return out_df


In [40]:
eval_df_simple = run_rag_hybrid_batch(
    df=df,
    index=index,
    encoder=encoder,
    bm25=bm25,
    tokenize_fn=tokenize_ru,
    sum_model=model,
    sum_tokenizer=tokenizer,
    items=items,
    k_retrieve=150,
    topN_each=1000,
    k_docs=30,
    snip_chars=800,
    max_new_tokens=2000,
    judge_llm=model,
    judge_tokenizer=tokenizer,
    judge_keep_threshold=1,
    judge_batch_size=32,
    gen_batch_size=8,
)


Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

...

Processed prompts:   0%|          | 0/8 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/8 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/8 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/6 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/6 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [42]:
eval_df_simple.to_parquet("eval_df_rag_filtered.parquet", index=False)